<a href="https://colab.research.google.com/github/itspritish/nlp/blob/master/ufit_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from fastai.text import *

In [0]:
path = untar_data(URLs.IMDB_SAMPLE)


In [0]:
df = pd.read_csv(path/'texts.csv')

In [0]:
data_lm = TextLMDataBunch.from_csv(path, 'texts.csv')

In [0]:
data_clas = TextClasDataBunch.from_csv(path, 'texts.csv', vocab=data_lm.train_ds.vocab, bs=32)

In [0]:
data_lm.save('data_lm_export.pkl')
data_clas.save('data_clas_export.pkl')

In [0]:
bs = 192

In [0]:
data_lm = load_data(path, 'data_lm_export.pkl', bs=bs)
data_clas = load_data(path, 'data_clas_export.pkl', bs=bs)

In [0]:
torch.cuda.set_device(0)

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.384619,3.978854,0.277902,00:09


In [0]:
learn.predict("This is a review about how", n_words=20)

'This is a review about how the Americans undoubtedly lost their courage to laugh , limited in comparison to their childhood . This is'

In [0]:
learn.save('ft')
learn.save_encoder('ft_enc')

In [0]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5).to_fp16()
learn.load_encoder('ft_enc')



In [0]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.675293,0.679502,0.537313,00:03


In [0]:
learn.unfreeze()
learn.fit_one_cycle(3, slice(1e-4, 1e-2))

epoch,train_loss,valid_loss,accuracy,time


RuntimeError: ignored

In [0]:
learn.predict("This was a great movie!")